# Neural Network SMS Text Classifier

In this project, the goal is to create a machine larning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

Now I will be using TensorFlow, because I don't have much experience on TorchText the PyTorch package for Natural Language processing

This same solution on Colab [here]("https://colab.research.google.com/drive/14tqM84g1w2LNFNFjRq4e8ddCuR2-Pcc_?usp=sharing") and more informations [here]("https://www.freecodecamp.org/learn/machine-learning-with-python/machine-learning-with-python-projects/neural-network-sms-text-classifier").

---

In [1]:
# import libraries
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from pathlib import Path

c:\Users\Lucas\AppData\Local\Programs\Python\data-science\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# get data files
data_path = Path("data")
sms_path = data_path / "sms"

if sms_path.is_dir():
    print(f"{sms_path} already exists, skipping download")
else:
    print(f"Creating {sms_path}")
    sms_path.mkdir(parents=True, exist_ok=True)

    with open(sms_path / 'train-data.tsv', 'wb') as f:
        print("Downloading train data")
        request = requests.get("https://cdn.freecodecamp.org/project-data/sms/train-data.tsv")
        f.write(request.content)
    
    with open(sms_path / "valid-data.tsv", 'wb') as f:
        print(f"Downloading validation data")
        request = requests.get("https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv")
        f.write(request.content)

data\sms already exists, skipping download


In [3]:
train_file_path = sms_path / "train-data.tsv"
test_file_path = sms_path / "valid-data.tsv"

train_file_path, test_file_path

(WindowsPath('data/sms/train-data.tsv'),
 WindowsPath('data/sms/valid-data.tsv'))

In [12]:
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])
train_df.dropna()
train_df.head()

,type,msg
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [13]:
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])
test_df.dropna()
train_df.head()

,type,msg
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [14]:
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]

In [15]:
train_df

,type,msg
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,0,just woke up. yeesh its late. but i didn't fal...
4175,0,what do u reckon as need 2 arrange transport i...
4176,1,free entry into our £250 weekly competition ju...
4177,1,-pls stop bootydelious (32/f) is inviting you ...


In [16]:
test_df

,type,msg
0,0,i am in hospital da. . i will return home in e...
1,0,"not much, just some textin'. how bout you?"
2,0,i probably won't eat at all today. i think i'm...
3,0,don‘t give a flying monkeys wot they think and...
4,0,who are you seeing?
...,...,...
1387,0,true dear..i sat to pray evening and felt so.s...
1388,0,"what will we do in the shower, baby?"
1389,0,where are you ? what are you doing ? are yuou ...
1390,1,ur cash-balance is currently 500 pounds - to m...


In [78]:
train_labels =  train_df["type"].values
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df["msg"].values, train_labels)
)

In [79]:
test_labels =  test_df["type"].values
test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df["msg"].values, test_labels)
)
test_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [80]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [83]:
vec = keras.layers.TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

vec.adapt(train_ds.map(lambda text, label: text))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [84]:
vocab = np.array(vec.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in', 'is',
       'me', 'my', 'for', 'your', 'of', 'it', 'call', 'have', 'on'],
      dtype='<U15')

In [85]:
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

In [86]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=15,
)

Epoch 1/15
131/131 [==============================] - 301s 2s/step - loss: 0.5793 - accuracy: 0.8660 - val_loss: 0.4545 - val_accuracy: 0.8604
Epoch 2/15
131/131 [==============================] - 264s 2s/step - loss: 0.3049 - accuracy: 0.8670 - val_loss: 0.1709 - val_accuracy: 0.8740
Epoch 3/15
131/131 [==============================] - 267s 2s/step - loss: 0.1208 - accuracy: 0.9660 - val_loss: 0.0886 - val_accuracy: 0.9760
Epoch 4/15
131/131 [==============================] - 266s 2s/step - loss: 0.0716 - accuracy: 0.9816 - val_loss: 0.0709 - val_accuracy: 0.9812
Epoch 5/15
131/131 [==============================] - 272s 2s/step - loss: 0.0574 - accuracy: 0.9859 - val_loss: 0.0633 - val_accuracy: 0.9854
Epoch 6/15
131/131 [==============================] - 271s 2s/step - loss: 0.0460 - accuracy: 0.9895 - val_loss: 0.0602 - val_accuracy: 0.9875
Epoch 7/15
131/131 [==============================] - 269s 2s/step - loss: 0.0352 - accuracy: 0.9923 - val_loss: 0.0661 - val_accuracy: 0.9865

In [87]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  ps = model.predict([pred_text])
  print(ps)
  p = ps[0][0]
  return [p, "ham" if p <0.5 else "spam"]

pred_text = "sale today! to stop texts call 98912460 4"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 7s 7s/step
[[2.2739158]]
[2.2739158, 'spam']


In [88]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 9891246032 4",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 [==============================] - 0s 304ms/step
[[-9.776394]]
1/1 [==============================] - 0s 242ms/step
[[2.2739158]]
1/1 [==============================] - 0s 201ms/step
[[-17.980015]]
1/1 [==============================] - 0s 218ms/step
[[5.8785124]]
1/1 [==============================] - 0s 200ms/step
[[6.204541]]
1/1 [==============================] - 0s 184ms/step
[[-14.803345]]
1/1 [==============================] - 0s 181ms/step
[[-17.708263]]
You passed the challenge. Great job!


**SUCCESS!!**

The model guessed correctly all texts!

This is the last course project on the FCC Machine Learning with Python.

***Thanks for everyone who read this far :)***